In [204]:
import pandas as pd
import numpy as np
from sklearn import linear_model
from sklearn.model_selection import train_test_split
from sklearn.svm import SVR



pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [122]:
data=pd.read_csv('chembl_24_bbb2.csv')
data.dropna(axis=1,how='all',inplace=True)
data

,molregno,pref_name,chembl_id,max_phase,therapeutic_flag,dosed_ingredient,structure_type,chebi_par_id,molecule_type,first_approval,oral,parenteral,topical,black_box_warning,natural_product,first_in_class,chirality,prodrug,inorganic_flag,usan_year,availability_type,usan_stem,polymer_flag,usan_substem,usan_stem_definition,indication_class,withdrawn_flag,withdrawn_year,withdrawn_country,withdrawn_reason,withdrawn_class,molregno-2,mw_freebase,alogp,hba,hbd,psa,rtb,ro3_pass,num_ro5_violations,acd_most_apka,acd_most_bpka,acd_logp,acd_logd,molecular_species,full_mwt,aromatic_rings,heavy_atoms,qed_weighted,mw_monoisotopic,full_molformula,hba_lipinski,hbd_lipinski,molregno-3,molfile,standard_inchi,standard_inchi_key,canonical_smiles,"Dissociation constant, pKa of the compound (value)","Dissociation constant, pKa of the compound (units)",Partition coefficient (logP) (value),Partition coefficient (logP) (units)
0,17017,"1,2-DICHLOROETHANE",CHEMBL16370,0,0,0,MOL,27789.0,Small molecule,NaN,0,0,0,0,-1,-1,-1,-1,-1,NaN,-1,NaN,0,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,17017,98.96,1.46,0,0,0.00,1,Y,0,NaN,NaN,1.65,1.65,NaN,98.96,0,4,0.44,97.9690,C2H4Cl2,0,0,17017,\n Mrv0541 04051115152D \n\n 4 3 ...,InChI=1S/C2H4Cl2/c3-1-2-4/h1-2H2,WSLDOOZREJYCGB-UHFFFAOYSA-N,ClCCCl,NaN,NaN,1.48,-
1,16832,BENZENE,CHEMBL277500,0,0,0,MOL,16716.0,Small molecule,NaN,0,0,0,0,-1,-1,-1,-1,-1,NaN,-1,NaN,0,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,16832,78.11,1.69,0,0,0.00,0,Y,0,NaN,NaN,2.18,2.18,NaN,78.11,1,6,0.44,78.0470,C6H6,0,0,16832,\n Mrv0541 04051115152D \n\n 6 6 ...,InChI=1S/C6H6/c1-2-4-6-5-3-1/h1-6H,UHOVQNZJYSORNB-UHFFFAOYSA-N,c1ccccc1,NaN,NaN,2.13,-
2,13440,BUTYL ALCOHOL,CHEMBL14245,0,0,1,MOL,28885.0,Small molecule,NaN,0,0,0,0,0,0,2,0,0,NaN,-1,NaN,0,NaN,NaN,Pharmaceutic Aid (solvent),0,NaN,NaN,NaN,NaN,13440,74.12,0.78,1,1,20.23,2,Y,0,NaN,NaN,0.84,0.84,NEUTRAL,74.12,0,5,0.51,74.0732,C4H10O,1,1,13440,\n Mrv0541 04051115152D \n\n 5 4 ...,"InChI=1S/C4H10O/c1-2-3-4-5/h5H,2-4H2,1H3",LRHPLDYGYMQRHN-UHFFFAOYSA-N,CCCCO,NaN,NaN,0.88,-
3,222293,OCTANE,CHEMBL134886,0,0,0,MOL,17590.0,Small molecule,NaN,0,0,0,0,-1,-1,-1,-1,-1,NaN,-1,NaN,0,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,222293,114.23,3.37,0,0,0.00,5,N,0,NaN,NaN,4.78,4.78,NaN,114.23,0,8,0.48,114.1409,C8H18,0,0,222293,\n Mrv0541 04051115152D \n\n 8 7 ...,"InChI=1S/C8H18/c1-3-5-7-8-6-4-2/h3-8H2,1-2H3",TVMXDCGIABBOFY-UHFFFAOYSA-N,CCCCCCCC,NaN,NaN,5.15,-
4,77128,BUPROPION,CHEMBL894,4,1,1,MOL,3219.0,Small molecule,1985.0,1,0,0,1,0,0,0,0,0,1974.0,1,NaN,0,NaN,NaN,Antidepressant,0,NaN,NaN,NaN,NaN,77128,239.75,3.30,2,1,29.10,3,N,0,NaN,7.16,2.32,2.13,NEUTRAL,239.75,1,16,0.82,239.1077,C13H18ClNO,2,1,77128,\n SciTegic12091219282D\n\n 16 16 0 0 0 0...,"InChI=1S/C13H18ClNO/c1-9(15-13(2,3)4)12(16)10-...",SNPPWIUOZRMYNY-UHFFFAOYSA-N,CC(NC(C)(C)C)C(=O)c1cccc(Cl)c1,15.43,-,NaN,NaN
5,13618,ALCOHOL,CHEMBL545,4,0,1,MOL,16236.0,Small molecule,NaN,0,1,1,0,0,0,2,0,0,NaN,2,NaN,0,NaN,NaN,"Pharmaceutic Aid (solvent); Anti-Infective, To...",0,NaN,NaN,NaN,NaN,13618,46.07,0.00,1,1,20.23,0,Y,0,NaN,NaN,-0.18,-0.18,NEUTRAL,46.07,0,3,0.41,46.0419,C2H6O,1,1,13618,\n Mrv0541 04051115152D \n\n 3 2 ...,"InChI=1S/C2H6O/c1-2-3/h3H,2H2,1H3",LFQSCWFLJHTTHZ-UHFFFAOYSA-N,CCO,16.00,-,-0.31,-
6,13617,METHYL ALCOHOL,CHEMBL14688,0,0,1,MOL,17790.0,Small molecule,NaN,0,0,0,0,0,0,2,0,0,NaN,-1,NaN,0,NaN,NaN,Pharmaceutic Aid (solvent),0,NaN,NaN,NaN,NaN,13617,32.04,-0.39,1,1,20.23,0,Y,0,NaN,NaN,-0.69,-0.69,NEUTRAL,32.04,0,2,0.39,32.0262,CH4O,1,1,13617,\n Mrv0541 04051115152D \n\n 2 1 ...,"InChI=1S/CH4O/c1-2/h2H,1H3",OKKJLVBELUTLKV-UHFFFAOYSA-N,CO,NaN,NaN,-0.77,-
7,238336,NITROGEN,CHEMBL142438,4,0,1,MOL,17997.0,Small molecule,2015.0,0,0,1,0,0,0,2,0,0,NaN,1,NaN,0,NaN,NaN,Pharmaceutic Aid (air displacement),0,NaN,NaN,NaN,NaN,238336,28.01,0.03,2,0,47.58,0,Y,0,NaN,NaN,-0.90,-0.90,NaN,28.01,0,2,0.35,28.0061,N2,2,0,238336,\n Mrv0541 04051115152D \n\n 2 1 ...,InChI=1S/N2/c1-2,IJGRMHOSHXDMSA-UHFFFAOYSA-N,N#N,NaN,NaN,NaN,NaN
8,13616,PROPANOL,CHEMBL14687,0,0,1,MOL,28831.0,Small molecule,NaN,0,0,0,0,0,0,2,0,0,NaN,

In [162]:
BBB=pd.read_csv('../../DrugDatabasesTools/BBB_Files/BBB2.csv')
BBB.drop(columns=['compounds','BBB_value','SMILES'],inplace=True)
BBB.drop_duplicates(inplace=True)
tt=BBB.canonical_smiles.value_counts()>1
tt
#Nc1nnc(-c2cccc(Cl)c2Cl)c(N)n1 True
#CC(Cl)(Cl)Cl True
#CN(C)CCCN1c2ccccc2CCc2ccccc21 True
#O=C1CC(=O)N(c2ccccc2)c2cc(Cl)ccc2N1 True

Nc1nnc(-c2cccc(Cl)c2Cl)c(N)n1                                                                                                                                                 True
CC(Cl)(Cl)Cl                                                                                                                                                                  True
CN(C)CCCN1c2ccccc2CCc2ccccc21                                                                                                                                                 True
O=C1CC(=O)N(c2ccccc2)c2cc(Cl)ccc2N1                                                                                                                                           True
CCCCCCCC1(CC)C(=O)NC(=O)NC1=O                                                                                                                                                False
CC(C)(C)/[N+]([O-])=C\c1ccccc1                                                                           

In [165]:
full=pd.merge(data, BBB, left_on='canonical_smiles', right_on='canonical_smiles')
full.head()

,molregno,pref_name,chembl_id,max_phase,therapeutic_flag,dosed_ingredient,structure_type,chebi_par_id,molecule_type,first_approval,oral,parenteral,topical,black_box_warning,natural_product,first_in_class,chirality,prodrug,inorganic_flag,usan_year,availability_type,usan_stem,polymer_flag,usan_substem,usan_stem_definition,indication_class,withdrawn_flag,withdrawn_year,withdrawn_country,withdrawn_reason,withdrawn_class,molregno-2,mw_freebase,alogp,hba,hbd,psa,rtb,ro3_pass,num_ro5_violations,acd_most_apka,acd_most_bpka,acd_logp,acd_logd,molecular_species,full_mwt,aromatic_rings,heavy_atoms,qed_weighted,mw_monoisotopic,full_molformula,hba_lipinski,hbd_lipinski,molregno-3,molfile,standard_inchi,standard_inchi_key,canonical_smiles,"Dissociation constant, pKa of the compound (value)","Dissociation constant, pKa of the compound (units)",Partition coefficient (logP) (value),Partition coefficient (logP) (units),BBB_classification
0,17017,"1,2-DICHLOROETHANE",CHEMBL16370,0,0,0,MOL,27789.0,Small molecule,NaN,0,0,0,0,-1,-1,-1,-1,-1,NaN,-1,NaN,0,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,17017,98.96,1.46,0,0,0.00,1,Y,0,NaN,NaN,1.65,1.65,NaN,98.96,0,4,0.44,97.9690,C2H4Cl2,0,0,17017,\n Mrv0541 04051115152D \n\n 4 3 ...,InChI=1S/C2H4Cl2/c3-1-2-4/h1-2H2,WSLDOOZREJYCGB-UHFFFAOYSA-N,ClCCCl,NaN,NaN,1.48,-,-
1,16832,BENZENE,CHEMBL277500,0,0,0,MOL,16716.0,Small molecule,NaN,0,0,0,0,-1,-1,-1,-1,-1,NaN,-1,NaN,0,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,16832,78.11,1.69,0,0,0.00,0,Y,0,NaN,NaN,2.18,2.18,NaN,78.11,1,6,0.44,78.0470,C6H6,0,0,16832,\n Mrv0541 04051115152D \n\n 6 6 ...,InChI=1S/C6H6/c1-2-4-6-5-3-1/h1-6H,UHOVQNZJYSORNB-UHFFFAOYSA-N,c1ccccc1,NaN,NaN,2.13,-,0
2,16832,BENZENE,CHEMBL277500,0,0,0,MOL,16716.0,Small molecule,NaN,0,0,0,0,-1,-1,-1,-1,-1,NaN,-1,NaN,0,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,16832,78.11,1.69,0,0,0.00,0,Y,0,NaN,NaN,2.18,2.18,NaN,78.11,1,6,0.44,78.0470,C6H6,0,0,16832,\n Mrv0541 04051115152D \n\n 6 6 ...,InChI=1S/C6H6/c1-2-4-6-5-3-1/h1-6H,UHOVQNZJYSORNB-UHFFFAOYSA-N,c1ccccc1,NaN,NaN,2.13,-,0
3,13440,BUTYL ALCOHOL,CHEMBL14245,0,0,1,MOL,28885.0,Small molecule,NaN,0,0,0,0,0,0,2,0,0,NaN,-1,NaN,0,NaN,NaN,Pharmaceutic Aid (solvent),0,NaN,NaN,NaN,NaN,13440,74.12,0.78,1,1,20.23,2,Y,0,NaN,NaN,0.84,0.84,NEUTRAL,74.12,0,5,0.51,74.0732,C4H10O,1,1,13440,\n Mrv0541 04051115152D \n\n 5 4 ...,"InChI=1S/C4H10O/c1-2-3-4-5/h5H,2-4H2,1H3",LRHPLDYGYMQRHN-UHFFFAOYSA-N,CCCCO,NaN,NaN,0.88,-,-
4,222293,OCTANE,CHEMBL134886,0,0,0,MOL,17590.0,Small molecule,NaN,0,0,0,0,-1,-1,-1,-1,-1,NaN,-1,NaN,0,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,222293,114.23,3.37,0,0,0.00,5,N,0,NaN,NaN,4.78,4.78,NaN,114.23,0,8,0.48,114.1409,C8H18,0,0,222293,\n Mrv0541 04051115152D \n\n 8 7 ...,"InChI=1S/C8H18/c1-3-5-7-8-6-4-2/h3-8H2,1-2H3",TVMXDCGIABBOFY-UHFFFAOYSA-N,CCCCCCCC,NaN,NaN,5.15,-,0


In [166]:
full['smiles_length'] = [len(i) for i in full['canonical_smiles']]
full['fullmolformula_length'] = [len(i) for i in full['full_molformula']]
full['smile_to_formula']=full['smiles_length']/full['fullmolformula_length']

def molecule_type_to_numeric(x):
    if x=='Small molecule':
        return 1
    else:
        return 0
full['molecule_type_nr'] = full['molecule_type'].apply(molecule_type_to_numeric)
full.drop(columns='molecule_type',inplace=True)

def y_n_to_nr(x):
    if x=='Y':
        return 1
    elif x=='N':
        return 0
    else:
        return np.nan
full['ro3_pass_nr'] = full['ro3_pass'].apply(y_n_to_nr)
full.drop(columns='ro3_pass',inplace=True)
def molecular_species_to_nr(x):
    if x=='NEUTRAL':
        return 1
    elif x=='BASE':
        return 2
    elif x=='ACID':
        return 3
    else:
        return np.nan
full['molecular_species_nr'] = full['molecular_species'].apply(molecular_species_to_nr)
full.drop(columns='molecular_species',inplace=True)
def BBB_classification_to_nr(x):
    if x=='-':
        return -1
    elif x=='0':
        return 0
    elif x=='+':
        return 1
    else:
        return np.nan
full['BBB_classification_nr'] = full['BBB_classification'].apply(BBB_classification_to_nr)
full.drop(columns='BBB_classification',inplace=True)

full.drop(columns=["molregno", "pref_name","chembl_id","chebi_par_id","standard_inchi_key","canonical_smiles",'usan_year',"structure_type",'usan_substem',"usan_stem_definition","withdrawn_country","withdrawn_reason","molregno-2",
                  "molregno-3","molfile","standard_inchi","molfile","standard_inchi","Dissociation constant, pKa of the compound (units)","Partition coefficient (logP) (units)","full_molformula",'indication_class'],inplace=True)

In [ ]:
full.withdrawn_class.unique()

In [202]:
full.dropna(axis=1,how='any',inplace=True)
full.isna().sum()>0#178

max_phase                                                        False
therapeutic_flag                                                 False
dosed_ingredient                                                 False
oral                                                             False
parenteral                                                       False
topical                                                          False
black_box_warning                                                False
natural_product                                                  False
first_in_class                                                   False
chirality                                                        False
prodrug                                                          False
inorganic_flag                                                   False
availability_type                                                False
polymer_flag                                                     False
withdr

category enconding, provided by [this](http://pbpython.com/categorical-encoding.html)  
cateogry way

In [184]:
full["usan_stem"] = full["usan_stem"].astype('category')
full["usan_stem_cat"] = full["usan_stem"].cat.codes
full.drop(columns='usan_stem',inplace=True)

in the same link, get dummy way

In [187]:
full=pd.get_dummies(full, columns=["withdrawn_class"])

KeyError: "['withdrawn_class'] not in index"

In [188]:
full.dtypes
full.drop(columns=['withdrawn_year','',''])

max_phase                                                          int64
therapeutic_flag                                                   int64
dosed_ingredient                                                   int64
first_approval                                                   float64
oral                                                               int64
parenteral                                                         int64
topical                                                            int64
black_box_warning                                                  int64
natural_product                                                    int64
first_in_class                                                     int64
chirality                                                          int64
prodrug                                                            int64
inorganic_flag                                                     int64
availability_type                                  

In [180]:
def check_correlation(df,target,corr_cutoff):
    data_train=df.sample(frac=0.7,random_state=200)
    data_test=df.drop(data_train.index)

    data_x=df.drop(columns=[target])
    data_y=df[target]

    data_x_train=data_train.drop(columns=[target])
    data_y_train=data_train[target]

    data_x_test=data_test.drop(columns=[target])
    data_y_test=data_test[target]
    
    corr=data_train.corr()
    #fig,ax=plt.subplots(figsize=(8,6))
    #sns.heatmap(corr)
    features=''
    features_text=''
    if len(corr[target].where(lambda x : x.abs()>corr_cutoff).dropna())>1:
        features=corr[target].where(lambda x : x.abs()>corr_cutoff).dropna()
        features_text=features.index.str.cat(sep=', ')+'\n'
    else:
        features='1'
        features_text='None'
    print('The features correlated with target above the threshold %s are %s' %(corr_cutoff,features_text))
    return len(features)

check_correlation(full,'BBB_classification_nr',0.5)

The features correlated with target above the threshold 0.5 are withdrawn_year, alogp, acd_most_apka, acd_logp, acd_logd, Dissociation constant, pKa of the compound (value), Partition coefficient (logP) (value), BBB_classification_nr



8

In [203]:
def xMlr(df,target,frac=0.7,cv=10):
    i=0
    mse=0
    score=0
    while i<cv:
        np.random.seed(seed=123)
        pka_data_train=df.sample(frac=0.7,random_state=200)
        pka_data_test=df.drop(pka_data_train.index)

        pka_data_x=df.drop(columns=[target])
        pka_data_y=df[target]

        pka_data_x_train=df.drop(columns=[target])
        pka_data_y_train=df[target]

        pka_data_x_test=df.drop(columns=[target])
        pka_data_y_test=df[target]
        regr = linear_model.LinearRegression()
        regr.fit(pka_data_x_train, pka_data_y_train)
        #print(regr.coef_)
        mse+=(np.mean((regr.predict( pka_data_x_test)-pka_data_y_test)**2))
        score+=regr.score(pka_data_x_test, pka_data_y_test)
        i+=1
    return mse/cv, score/cv
mlr_mse,mlr_score=xMlr(full,'BBB_classification_nr')
print("RMSE is %s. Score is %s." % (mlr_mse, mlr_score))

RMSE is 0.14361951052473237. Score is 0.5879716976217296.


In [206]:
def xSVR(df,target,frac=0.7,cv=10):
    i=0
    mse=0
    score=0
    while i<cv:
        np.random.seed(seed=123)
        pka_data_train=df.sample(frac=0.7,random_state=200)
        pka_data_test=df.drop(pka_data_train.index)

        pka_data_x=df.drop(columns=[target])
        pka_data_y=df[target]

        pka_data_x_train=df.drop(columns=[target])
        pka_data_y_train=df[target]

        pka_data_x_test=df.drop(columns=[target])
        pka_data_y_test=df[target]
        clf = SVR(gamma='scale', C=1.0, epsilon=0.1)
        clf.fit(pka_data_x_train, pka_data_y_train) 
        mse+=(np.mean((clf.predict( pka_data_x_test)-pka_data_y_test)**2))
        score+=clf.score(pka_data_x_test, pka_data_y_test, sample_weight=None)
        i+=1
    return mse/cv, score/cv
svr_mse,svr_score=xSVR(full,'BBB_classification_nr')
print("MSE is %s. Score is %s." % (svr_mse, svr_score))

MSE is 0.15334520804700585. Score is 0.5600697598912231.
